In [3]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('/content/data.db')

query_tables = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(query_tables, conn)

tables

,name
0,balances
1,client_products
2,client
3,inv_campaign_eval


## BALANCE TABLE - cleaning:

In [4]:
# Preview the first few rows
balances_preview = pd.read_sql_query("SELECT * FROM balances LIMIT 5;", conn)
balances_preview


,date,balance,client_id,currency
0,2023-06-06 16:21:15.291346,2680.467773,249789938,CZK
1,2023-05-23 16:21:15.291346,2654.055419,249789938,CZK
2,2023-05-09 16:21:15.291346,2620.835872,249789938,CZK
3,2023-04-25 16:21:15.291346,2720.315925,249789938,CZK
4,2023-04-11 16:21:15.291346,2687.601982,249789938,CZK


In [5]:
# Check for missing values in each column
query_missing_values = """
SELECT
    (SELECT COUNT(*) FROM balances) - COUNT(date) AS missing_date,
    (SELECT COUNT(*) FROM balances) - COUNT(balance) AS missing_balance,
    (SELECT COUNT(*) FROM balances) - COUNT(client_id) AS missing_client_id,
    (SELECT COUNT(*) FROM balances) - COUNT(currency) AS missing_currency
FROM balances;
"""

# Check for unique currencies
query_currency_consistency = "SELECT DISTINCT currency FROM balances;"

missing_values = pd.read_sql_query(query_missing_values, conn)
currency_consistency = pd.read_sql_query(query_currency_consistency, conn)

missing_values, currency_consistency



(   missing_date  missing_balance  missing_client_id  missing_currency
 0             0                0                  0                 0,
   currency
 0      CZK
 1      USD
 2      EUR)

- RESULT: there are three currencies: CZK, EUR, USD
> ??? change data to one currency



In [12]:
# check the data types
sample_data_types_balances = balances_preview.dtypes

sample_data_types_balances

date          object
balance      float64
client_id      int64
currency      object
dtype: object

- RESULT: appropriate data types



In [8]:
# Identify duplicate records in the 'balances' table based on 'client_id', 'balance', 'currency', and 'date'
duplicate_balances_query = """
SELECT client_id, date, balance, currency, COUNT(*) as duplicate_count
FROM balances
GROUP BY client_id, date, balance, currency
HAVING COUNT(*) > 1;
"""

duplicate_balances = pd.read_sql_query(duplicate_balances_query, conn)

# Display the number of duplicate records found and the first few for review
duplicate_balances_count = len(duplicate_balances)
duplicate_balances_count, duplicate_balances.head()




(0,
 Empty DataFrame
 Columns: [client_id, date, balance, currency, duplicate_count]
 Index: [])

- RESULT: there are no duplicates

## CLIENT table - cleaning:

In [9]:
# Preview the first few rows
client_preview = pd.read_sql_query("SELECT * FROM client LIMIT 5;", conn)
client_preview


,client_id,age,job,marital,education,gender
0,249789938,38,services,married,secondary,M
1,1222646323,46,services,divorced,unknown,M
2,451375919,33,admin.,single,secondary,F
3,338972671,44,self-employed,married,secondary,F
4,1472834688,36,blue-collar,married,primary,M


In [21]:
df_client = pd.read_sql_query("SELECT * FROM client;", conn)

In [10]:
# Check for missing values
query_missing_values_client = """
SELECT
    (SELECT COUNT(*) FROM client) - COUNT(client_id) AS missing_client_id,
    (SELECT COUNT(*) FROM client) - COUNT(age) AS missing_age,
    (SELECT COUNT(*) FROM client) - COUNT(job) AS missing_job,
    (SELECT COUNT(*) FROM client) - COUNT(marital) AS missing_marital,
    (SELECT COUNT(*) FROM client) - COUNT(education) AS missing_education,
    (SELECT COUNT(*) FROM client) - COUNT(gender) AS missing_gender
FROM client;
"""

# Validate data types by showing a sample of each column
sample_data_types_client = client_preview.dtypes

missing_values_client = pd.read_sql_query(query_missing_values_client, conn)
missing_values_client, sample_data_types_client


(   missing_client_id  missing_age  missing_job  missing_marital  \
 0                  0          232          438                0   
 
    missing_education  missing_gender  
 0                  0               0  ,
 client_id     int64
 age           int64
 job          object
 marital      object
 education    object
 gender       object
 dtype: object)

- RESULT: there are missing values in columns AGE and JOB

> ??? AGE: use MEDIAN to input missing values
> ??? JOB: instead of NULL input "unknown"





In [14]:
# Check for upper case values in categorical columns of the 'client' table
uppercase_counts = {
    column: df_client[column].str.contains(r'[A-Z]').sum()
    for column in ['job', 'marital', 'education', 'gender']
}

uppercase_counts


{'job': 0, 'marital': 0, 'education': 0, 'gender': 11162}

- RESULT: no need to lower any uppercase letters, the Gender column is uppercase on purpose

In [20]:
# check for duplicate records
duplicates = df_client[df_client.duplicated(subset='client_id', keep=False)]
if not duplicates.empty:
    df_client = df_client.drop_duplicates(subset='client_id', keep='first')

number_of_duplicates = duplicates.sum()
number_of_duplicates


client_id    0.0
age          0.0
job          0.0
marital      0.0
education    0.0
gender       0.0
dtype: float64

- RESULT: there arent any duplicates

In [22]:
# Validate and correct gender values

unique_genders = df_client['gender'].unique()

# Identify invalid gender values by filtering out the valid ones
invalid_genders = [gender for gender in unique_genders if gender not in ['M', 'F']]


# Count the number of records with invalid gender values
num_invalid_gender_records = df_client['gender'].isin(invalid_genders).sum()
num_invalid_gender_records


0

- RESULT: there arent any invalid genders

## CLIENT PRODUCTS - cleaning:

In [24]:
client_products_preview = pd.read_sql_query("SELECT * FROM client_products LIMIT 5;", conn)
client_products_preview

,client_id,has_deposits,loan,has_insurance,has_mortgage
0,249789938,yes,no,n,no
1,1222646323,no,no,n,no
2,451375919,no,no,n,yes
3,338972671,no,no,y,yes
4,1472834688,yes,no,n,yes
